In [552]:
np.array([[[1,2],[3,4]],[[5,6],[7,8]]]).shape

(2, 2, 2)

In [573]:
np.random.randint(5, 11, size=(2,3))

array([[10,  6,  5],
       [ 5,  8,  9]])

In [575]:
np.random.random((2,3))

array([[0.21350669, 0.30176854, 0.3524159 ],
       [0.52568785, 0.82339817, 0.45256408]])

In [576]:
np.zeros((3,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [582]:
np.ones((3,3))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [585]:
np.identity(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [586]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [622]:
x = (2 > 3)
x

False

In [673]:
trials = []
for i in range(100):
    arr = np.random.randint(10, size=(4,4))
    tup = len(arr[arr>5]), len(arr[arr<=5])
    trials.append(tup)

trials

truths = []
for trial in trials:
    truth = ( trial[1] > trial[0])
    truths.append(truth)

sum(truths)

71

#### Import all modules:

In [200]:
import regex as re
import slate3k
import pandas as pd
import numpy as np
import string
import requests
from bs4 import BeautifulSoup

from os import listdir, rename
from os.path import isfile, join

#### List of files

In [207]:
mypath = '/home/dretoabasi/portfolio/NCDC/pdf_files'

filelist = sorted([join(mypath, file) for file in listdir(mypath) if isfile(join(mypath, file))])

#### Read in the pdf from disk

In [2]:
def read_in_pdf_file_from_disk(filename):
    with open(filename, 'rb') as f:
        pdftext = slate3k.PDF(f)
        pdftext = list(pdftext)
        print("file reading and parsing: SUCCESSFUL")
        print("There are **{}** pages in this pdf".format(len(pdftext)))
        return pdftext

In [206]:
pdftext = read_in_pdf_file_from_disk(filelist[113])

file reading and parsing: SUCCESSFUL
There are **5** pages in this pdf


#### Get the vital page: Current_text

In [429]:
def get_current_text(pdftext, filelist_index):
    for page in pdftext:
        if filelist_index > 34:
            temp = re.search(r'\s*GENERAL ?FACT ?SHEET', page)
        else:
            temp = re.search(r' Summary of COVID-19 cases in Nigeria  \n\nStates', page)
        if temp:
            page = page.replace(',', '').replace('FCT', 'Abuja')
            if filelist_index == 97:
                page = re.sub(r'(\w+)(\n\n)', '\g<1> \g<2>', page)
            return page

In [324]:
current_text = get_current_text(pdftext)

current_text

 Get the <b>date</b> of report

In [ ]:
34 ==> '\n\nAfrica (As at 5th April 2020) '
30 ==> '\n\nGlobal (As at 1st April 2020) '

In [436]:
def get_date(current_text, filelist_index):
    if filelist_index == 97:
        date = '20200607'
        return date
    if filelist_index >54:
        date = re.search(r'(?<=DATA ?AS ?AT ?)\w+[ \w]+\d+(?=\s*)', current_text).group()
    elif 35 <= filelist_index <= 54:
        date = re.search(r'(?<=SHEET\s+\()\w+[ \w]+\d+(?=\)\s+)', current_text).group()
    elif filelist_index <= 34:
        date = re.search(r'(?<=\s+\w+\s+\([ a-zA-Z]+)\d+[ \w]+\d+(?=\)\s*)', current_text).group()
        
    date = pd.to_datetime(date).strftime('%Y%m%d')
    return date

Get the 6 <b>Geopolitical zones</b>

In [18]:
response = requests.get("https://en.wikipedia.org/wiki/Geopolitical_zones_of_Nigeria")
print("Status code is {}".format(response.status_code))
webpage = response.content

Status code is 200


In [42]:
soup = BeautifulSoup(webpage, "html.parser")

In [87]:
main_tag = soup.find_all('div', class_="div-col columns column-width".split())[0].contents[1]

In [407]:
region_and_states = {}

for li_tag in main_tag.find_all('li', recursive=False):
    region = li_tag.b.get_text()
    state_list = li_tag.ul.get_text().split('\n')
    for state in state_list:
        state = state.replace('Federal Capital Territory','Abuja') if state == 'Federal Capital Territory' else state
        region_and_states[state] = region

region_and_states

{'Benue': 'North Central',
 'Kogi': 'North Central',
 'Kwara': 'North Central',
 'Nasarawa': 'North Central',
 'Niger': 'North Central',
 'Plateau': 'North Central',
 'Abuja': 'North Central',
 'Adamawa': 'North East',
 'Bauchi': 'North East',
 'Borno': 'North East',
 'Gombe': 'North East',
 'Taraba': 'North East',
 'Yobe': 'North East',
 'Jigawa': 'North West',
 'Kaduna': 'North West',
 'Kano': 'North West',
 'Katsina': 'North West',
 'Kebbi': 'North West',
 'Sokoto': 'North West',
 'Zamfara': 'North West',
 'Abia': 'South East',
 'Anambra': 'South East',
 'Ebonyi': 'South East',
 'Enugu': 'South East',
 'Imo': 'South East',
 'Akwa Ibom': 'South South',
 'Bayelsa': 'South South',
 'Cross River': 'South South',
 'Rivers': 'South South',
 'Delta': 'South South',
 'Edo': 'South South',
 'Ekiti': 'South West',
 'Lagos': 'South West',
 'Ogun': 'South West',
 'Ondo': 'South West',
 'Osun': 'South West',
 'Oyo': 'South West'}

Get the <b>Pattern</b>

In [ ]:
" \n\nOsun \n\n67 \n\n7 \n\n46 \n\n0 \n\n5 \n\n0 \n\n16 \n\n692 \n\n0"

In [765]:
re.sub(r'.*\s+Lagos', 'Lagos', list_[0][0], re.S)

'Lagos'

In [754]:
list_ = get_pattern(34).findall(current_text)
list_[0][0]

'Deaths \n\nLagos'

In [728]:
def get_pattern(filelist_index):
    if filelist_index > 34: #(34 < filelist_index < 100):
        pattern = re.compile(r'\s+([A-Z][a-z]+\s*[a-zA-Z]+) *((?:\s+\d+){7,8}\s+(?:\d+|N/A))')
    elif filelist_index < 35:
        pattern = re.compile(r'\s+([A-Z][a-z]+\s*[a-zA-Z]+) *((?:\s+\d+){3}\s+(?:\d+|N/A))')
#     elif filelist_index > 99:
#         pattern = re.compile(r'\s+([A-Z][a-z]+\s*[a-zA-Z]+) *((?:\s+\d+){7,8}\s+(?:\d+|N/A))')
    
    return pattern

In [442]:
pattern.findall(current_text)

[('Lagos', '\n\n163 \n\n8 \n\n46 \n\n7 \n\n2 \n\n0 \n\n118 \n\n0'),
 ('Abuja', '\n\n56 \n\n2 \n\n7 \n\n0 \n\n2 \n\n0 \n\n47 \n\n0'),
 ('Osun', '\n\n20 \n\n0 \n\n1 \n\n0 \n\n0 \n\n0 \n\n19 \n\n7'),
 ('Edo', '\n\n12 \n\n0 \n\n0 \n\n0 \n\n1 \n\n0 \n\n11 \n\n2'),
 ('Oyo', '\n\n11 \n\n0 \n\n0 \n\n0 \n\n0 \n\n0 \n\n11 \n\n3'),
 ('Bauchi', '\n\n6 \n\n0 \n\n0 \n\n0 \n\n0 \n\n0 \n\n6 \n\n2'),
 ('Kaduna', '\n\n6 \n\n1 \n\n0 \n\n0 \n\n0 \n\n0 \n\n6 \n\n0'),
 ('Akwa Ibom', '\n\n5 \n\n0 \n\n0 \n\n0 \n\n0 \n\n0 \n\n5 \n\n9'),
 ('Ogun', '\n\n7 \n\n0 \n\n2 \n\n0 \n\n0 \n\n0 \n\n2 \n\n10'),
 ('Katsina', '\n\n4 \n\n3 \n\n0 \n\n0 \n\n1 \n\n0 \n\n3 \n\n0'),
 ('Ekiti', '\n\n2 \n\n0 \n\n1 \n\n0 \n\n0 \n\n0 \n\n1 \n\n9'),
 ('Enugu', '\n\n2 \n\n0 \n\n0 \n\n0 \n\n0 \n\n0 \n\n2 \n\n14'),
 ('Rivers', '\n\n2 \n\n0 \n\n1 \n\n0 \n\n0 \n\n0 \n\n1 \n\n4'),
 ('Kwara', '\n\n2 \n\n0 \n\n0 \n\n0 \n\n0 \n\n0 \n\n2 \n\n4'),
 ('Delta', '\n\n2 \n\n0 \n\n0 \n\n0 \n\n1 \n\n0 \n\n1 \n\n1'),
 ('Ondo', '\n\n2 \n\n1 \n\n0 \n\n0 \n

Get the <b>Figures</b>

In [446]:
re.split(r'\D+','\n\n163 \n\n8 \n\n46 \n\n7 \n\n2 \n\n0 \n\n118 \n\n0'.strip())

['163', '8', '46', '7', '2', '0', '118', '0']

In [720]:
def get_figures(data, filelist_index):
    
    figures = []

    for datum in data:
        daily_figures = re.split(r'\D+', datum[1].strip())
        if len(daily_figures) == 8: #(34 < filelist_index < 100):
            daily_figures.append(np.nan)
        elif filelist_index < 35:
            daily_figures[1] = np.nan
            daily_figures.insert(3, np.nan)
            daily_figures.extend([np.nan]*4)
        elif (filelist_index > 99) and (filelist_index not in [106]):
            daily_figures = np.array(daily_figures)
            daily_figures[[7,8]] = daily_figures[[8,7]] #if filelist_index not in [106] else daily_figures
            daily_figures = daily_figures.tolist()
        
        figures.append(daily_figures)
    return figures

In [515]:
test = [1, 'a', 78, 10]
test[1] = np.nan
test.insert(3, np.nan)
test.extend([np.nan]*4)
test

[1, nan, 78, nan, 10, nan, nan, nan, nan]

In [521]:
test = np.array([1, 'a', 78, 10])
test[[2,3]] = test[[3,2]]
# test = list(np.array(test)[[0,1,3,2]])
list(test)

['1', 'a', '10', '78']

In [ ]:
figure filelist -- < 34  None[1,3]
'''
1 No. of Cases (Lab Confirmed) === TOTAL CONFIRMED CASES   0
2 No. of Cases (on admission) === 
3 No. Discharged ==== TOTAL DISCHARGED CASES   2
4 No of Deaths ==== TOTAL DEATHS   4
'''

In [ ]:
figures filelist 35 -- 99
'''
0 TOTAL CONFIRMED CASES
1 CONFIRMED NEW CASES
2 TOTAL DISCHARGED CASES
3 NEW DISCHARGED CASES
4 TOTAL DEATHS
5 NEW DEATHS
6 TOTAL ACTIVE CASES
7 DAYS SINCE LAST REPORTED CASE
'''
figures filelist 100 --'''
8 NO OF TESTS DONE'''

Assemble into <b>dataframe</b>

In [768]:
def create_df(current_text, region_and_states, pattern, filelist_index):
    df1 = pd.DataFrame(columns=['Date',
                                'State',
                                'Geo_pol_zone'
                               ])
    
    df2 = pd.DataFrame(columns=['Tot_Conf','New_Conf','Tot_disch','New_disch','Tot_Deaths',
                                'New_Deaths', 'Tot_Active', 'days_slrc', 'Tot_num_tests'
                               ])
    
    pattern = get_pattern(filelist_index)
    data = pattern.findall(current_text)
    date = get_date(current_text, filelist_index)
    figures = get_figures(data, filelist_index)
    index = 1

    
    for datum in data:
        state = re.sub(r'\s+', ' ', re.sub(r'.*\s+Lagos', 'Lagos', datum[0]))
        if region_and_states.get(state):
            df1.loc[index] = [date,
                              state,
                              region_and_states.get(state)
                             ]
            df2.loc[index] = figures[index-1]
        index += 1
            
    dframe = pd.concat([df1, df2], axis=1)  #df1.append(df2)
        
    return dframe

In [529]:
pd.concat?

In [ ]:
figure filelist -- < 34
'''
1 No. of Cases (Lab Confirmed) === TOTAL CONFIRMED CASES   1
2 No. of Cases (on admission) === 
3 No. Discharged ==== TOTAL DISCHARGED CASES   3
4 No of Deaths ==== TOTAL DEATHS   5
'''

In [ ]:
figures filelist 35 -- 99
'''
1 TOTAL CONFIRMED CASES
2 CONFIRMED NEW CASES
3 TOTAL DISCHARGED CASES
4 NEW DISCHARGED CASES
5 TOTAL DEATHS
6 NEW DEATHS
7 TOTAL ACTIVE CASES
8 DAYS SINCE LAST REPORTED CASE
'''
figures filelist 100 --'''
9 NO OF TESTS DONE'''


In [783]:
!howdoi except all errors python

try:
    do_something()
except:
    print "Caught it!"


In [784]:
error_list = []
for filelist_index, file in enumerate(filelist):
    try:
        pdftext = read_in_pdf_file_from_disk(file)
        current_text = get_current_text(pdftext, filelist_index)
        dframe = create_df(current_text, region_and_states, pattern, filelist_index)
    except:#(TypeError, AttributeError, ValueError):
        error_list.append(file)

file reading and parsing: SUCCESSFUL
There are **2** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **2** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **2** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **2** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **2** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **2** pages in this pdf
file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf
file r

file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **3** pages in this pdf


...

file reading and parsing: SUCCESSFUL
There are **5** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **5** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **4** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **4** pages in this pdf


file reading and parsing: SUCCESSFUL
There are **5** pages in this pdf


In [786]:
error_list

['/home/dretoabasi/portfolio/NCDC/pdf_files/file_001.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_002.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_003.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_004.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_005.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_006.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_007.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_008.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_009.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_010.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_011.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_012.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_013.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_014.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_015.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_016.pdf',
 '/home/dretoabasi/portfolio/NCDC/pdf_files/file_017.pdf

In [715]:
107 not in [106]

True

In [701]:
get_date(current_text, filelist_index)

'20200611'

In [770]:
current_text

'COVID-19 Situation Report  \n\n2 | P a g e \n\n198 \n\nAfrica (As at 5th April 2020) 1 \n\nAfrican Countries with reported cases  \n\nNew \n\nCountries  \n\n(Within Last 24hrs) \n\nTotal \n\nCountries  \n\nAffected (N) \n\nAlgeria Angola Benin Botswana Burundi Burkina Faso Cabo Verde \n\nCameroon Chad Central African Republic Congo Cote D’Ivoire \n\nDemocratic Republic of Congo \n\n(DRC) Djibouti Egypt Equatorial \n\nGuinea Eritrea Eswatini Ethiopia Gabon Gambia Ghana Guinea \n\nKenya Liberia Madagascar Malawi Mauritania Mauritius Morocco \n\nMozambique Namibia Niger Nigeria Rwanda Senegal Seychelles \n\nSierra Leone Somalia South Africa Sudan Tanzania Togo Tunisia \n\nUganda Zambia and Zimbabwe \n\n- \n\n 49(54) * \n\n* 54 African countries \n\nTable 2: Summary of COVID-19 cases in Nigeria  \n\nStates Affected \n\nNo. of Cases (Lab Confirmed) \n\nNo. of Cases (on admission) \n\nNo. Discharged \n\nNo of Deaths \n\nLagos \n\n120 \n\n91 \n\n30 \n\n2 \n\nAbuja \n\n47 \n\n43 \n\n0 \n\n2 \

In [540]:
re.search(r'(?<=SHEET\s+\()\w+[ \w]+\d+(?=\)\s+)', current_text).group()

AttributeError: 'NoneType' object has no attribute 'group'

In [291]:
54 ==> '\n\nGENERAL FACT SHEET (April 25th 2020) \n\nSTATES'
50 ==> '\n\nGENERAL FACT SHEET (April 21st 2020) \n\nSTATES'

In [402]:
current_text

In [238]:
bool(109)

True